# Problem 3 - Ray Tune for Hyperparameter Optimization

Sources: https://docs.ray.io/en/latest/tune/index.html

## 3.1

In [ ]:
#!pip install -U "ray[tune]"

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

import ray
from ray import tune
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.schedulers import ASHAScheduler
from ray.air import session
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.schedulers import HyperBandScheduler
from ray.tune.search.hyperopt import HyperOptSearch

import time
import numpy as np


In [ ]:
# Initialize Ray
ray.init(ignore_reinit_error=True)

2023-12-05 01:02:30,639	INFO worker.py:1673 -- Started a local Ray instance.


Python version:,3.8.6
Ray version:,2.8.1


### Grid Search

In [ ]:
def train_mnist(config):
    num_classes = 10
    epochs = 12

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train.reshape(-1, 28, 28, 1) / 255.0, x_test.reshape(-1, 28, 28, 1) / 255.0
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=config["conv_filters"], kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(config["dropout"]),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"]),
        metrics=["accuracy"])

    for epoch in range(epochs):
        model.fit(
            x_train,
            y_train,
            batch_size=config["batch_size"],
            epochs=1,
            verbose=0,
            validation_data=(x_test, y_test))

        # Evaluate the model
        _, accuracy = model.evaluate(x_test, y_test, verbose=0)
        session.report({"mean_accuracy": accuracy})


In [ ]:
start_time = time.time()

grid_analysis = tune.run(
    train_mnist,
    name="exp",
    metric="mean_accuracy",
    mode="max",
    stop={"mean_accuracy": 0.99},
    resources_per_trial={"gpu": 1},
    config={
        "conv_filters": tune.grid_search([64, 128, 256]),
        "lr": tune.grid_search([0.001, 0.01, 0.1]),
        "batch_size": tune.grid_search([64, 128, 256]),
        "dropout": tune.grid_search([0.0, 0.25, 0.5, 0.75, 0.9]),
    }
)

end_time = time.time()
grid_time = end_time - start_time

2023-11-24 16:00:22,466	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(pid=3069754) 2023-11-24 16:00:31.576563: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=3069754) 2023-11-24 16:00:31.623251: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=3069754) To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3069754) 2023-11-24 16:00:32.263246: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(train_mnist pid=3069754) 2023-11-24 16:00:36.468520: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  

Trial name,mean_accuracy
train_mnist_7b9c5_00000,0.9864
train_mnist_7b9c5_00001,0.9864
train_mnist_7b9c5_00002,0.9873
train_mnist_7b9c5_00003,0.9876
train_mnist_7b9c5_00004,0.9846
train_mnist_7b9c5_00005,0.9872
train_mnist_7b9c5_00006,0.9855
train_mnist_7b9c5_00007,0.9859
train_mnist_7b9c5_00008,0.9844
train_mnist_7b9c5_00009,0.9894


(pid=3070976) 2023-11-24 16:01:12.884145: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=3070976) 2023-11-24 16:01:12.930691: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=3070976) To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3070976) 2023-11-24 16:01:13.562473: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(train_mnist pid=3070976) 2023-11-24 16:01:14.925942: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  

### Beyesian Search

In [ ]:
#!pip install bayesian-optimization #--yes

In [ ]:
def train_mnist(config):
    # Convert continuous parameter back to categorical
    batch_size_map = {0: 64, 1: 128, 2: 256}

    config["conv_filters"] = int(round(config["conv_filters"]))
    config["batch_size"] = batch_size_map[int(round(config["batch_size"]))]

    # Load MNIST data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train.reshape(-1, 28, 28, 1) / 255.0, x_test.reshape(-1, 28, 28, 1) / 255.0

    # Define the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=config["conv_filters"], kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(config["dropout"]),
        tf.keras.layers.Dense(10, activation="softmax")  # Assuming 10 classes for MNIST
    ])

    # Compile the model
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"]),
        metrics=["accuracy"]
    )

    # Train the model
    model.fit(
        x_train,
        y_train,
        batch_size=config["batch_size"],
        epochs=12,  # or any other number of epochs you wish to use
        verbose=1,
        validation_data=(x_test, y_test)
    )

    # Evaluate the model
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    session.report({"mean_accuracy": accuracy})

In [ ]:
# Define the modified search space
search_space = {
    "conv_filters": tune.uniform(64,256),
    "lr": tune.loguniform(0.001, 0.1),
    "batch_size": tune.uniform(0, 2),  # 0 for 64, 1 for 128, 2 for 256
    "dropout": tune.uniform(0, 1)
}

# Initialize Bayesian optimization search algorithm
bayesopt_search = BayesOptSearch()

# Run the optimization using Bayesian search
start_time = time.time()

bayes_analysis = tune.run(
    train_mnist,
    name="exp_bayes",
    metric="mean_accuracy",
    mode="max",
    stop={"mean_accuracy": 0.99},
    resources_per_trial={"gpu": 1},
    config=search_space,
    search_alg=bayesopt_search,
    num_samples=10
)

end_time = time.time()
bayes_time = end_time - start_time

2023-12-05 01:39:33,602	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-12-05 01:39:33,620	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform sampler will be dropped.


(pid=3295966) 2023-12-05 01:39:35.572982: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=3295966) 2023-12-05 01:39:35.575219: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
(pid=3295966) 2023-12-05 01:39:35.620141: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
(pid=3295966) 2023-12-05 01:39:35.620637: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=3295966) To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3295966) 2023-12-05 01:39:3

(train_mnist pid=3295966) Epoch 1/12
469/469 [==============================] - 16s 32ms/step - loss: 2.7789 - accuracy: 0.1061 - val_loss: 2.3039 - val_accuracy: 0.1135
(train_mnist pid=3295966) Epoch 2/12
469/469 [==============================] - 15s 31ms/step - loss: 2.3045 - accuracy: 0.1055 - val_loss: 2.3064 - val_accuracy: 0.1135
(train_mnist pid=3295966) Epoch 3/12
469/469 [==============================] - 15s 31ms/step - loss: 2.3050 - accuracy: 0.1041 - val_loss: 2.3058 - val_accuracy: 0.1135
(train_mnist pid=3295966) Epoch 4/12
469/469 [==============================] - 15s 31ms/step - loss: 2.3048 - accuracy: 0.1061 - val_loss: 2.3062 - val_accuracy: 0.1135
(train_mnist pid=3295966) Epoch 5/12
469/469 [==============================] - 15s 31ms/step - loss: 2.3056 - accuracy: 0.1067 - val_loss: 2.3055 - val_accuracy: 0.1009
(train_mnist pid=3295966) Epoch 6/12
469/469 [==============================] - 15s 31ms/step - loss: 2.3051 - accuracy: 0.1064 - val_loss: 2.3075 - v

Trial name,mean_accuracy
train_mnist_1a912c10,0.9793
train_mnist_40c5a41c,0.8942
train_mnist_65d41f89,0.9508
train_mnist_a60fcaaf,0.9616
train_mnist_b9914140,0.862
train_mnist_cd0082ba,0.9597
train_mnist_eecf567c,0.1135
train_mnist_f2b95443,0.9085
train_mnist_fb005315,0.1135
train_mnist_ffca9cac,0.9812


(pid=3296841) 2023-12-05 01:42:38.962325: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=3296841) 2023-12-05 01:42:38.964613: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
(pid=3296841) 2023-12-05 01:42:39.013146: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
(pid=3296841) 2023-12-05 01:42:39.013681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=3296841) To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3296841) 2023-12-05 01:42:3

...

(pid=3301447) 2023-12-05 01:55:25.084528: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=3301447) 2023-12-05 01:55:25.133737: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=3301447) To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3301447) 2023-12-05 01:55:25.133233: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used. [repeated 2x across cluster]
(pid=3301447) 2023-12-05 01:55:26.028646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(pid=3301447) /

(train_mnist pid=3301447) Epoch 1/12
938/938 [==============================] - 19s 20ms/step - loss: 3.5198 - accuracy: 0.1044 - val_loss: 2.3108 - val_accuracy: 0.0892
(train_mnist pid=3301447) Epoch 2/12
938/938 [==============================] - 19s 20ms/step - loss: 2.3080 - accuracy: 0.1041 - val_loss: 2.3025 - val_accuracy: 0.1135
(train_mnist pid=3301447) Epoch 3/12
938/938 [==============================] - 19s 20ms/step - loss: 2.3083 - accuracy: 0.1051 - val_loss: 2.3046 - val_accuracy: 0.1010
(train_mnist pid=3301447) Epoch 4/12
938/938 [==============================] - 19s 20ms/step - loss: 2.3082 - accuracy: 0.1050 - val_loss: 2.3129 - val_accuracy: 0.1135
(train_mnist pid=3301447) Epoch 5/12
938/938 [==============================] - 19s 20ms/step - loss: 2.3087 - accuracy: 0.1032 - val_loss: 2.3074 - val_accuracy: 0.0980
(train_mnist pid=3301447) Epoch 6/12
938/938 [==============================] - 19s 20ms/step - loss: 2.3083 - accuracy: 0.1049 - val_loss: 2.3152 - v

(pid=3302450) 2023-12-05 01:59:15.119165: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=3302450) 2023-12-05 01:59:15.167325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=3302450) To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3302450) 2023-12-05 01:59:15.166816: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used. [repeated 2x across cluster]
(pid=3302450) 2023-12-05 01:59:16.068071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(pid=3302450) /

(train_mnist pid=3302450) Epoch 1/12
469/469 [==============================] - 6s 11ms/step - loss: 0.9973 - accuracy: 0.7110 - val_loss: 0.3906 - val_accuracy: 0.8828
(train_mnist pid=3302450) Epoch 2/12
469/469 [==============================] - 5s 10ms/step - loss: 0.6770 - accuracy: 0.7885 - val_loss: 0.3305 - val_accuracy: 0.9015
(train_mnist pid=3302450) Epoch 3/12
469/469 [==============================] - 5s 10ms/step - loss: 0.6658 - accuracy: 0.7930 - val_loss: 0.3305 - val_accuracy: 0.9054
(train_mnist pid=3302450) Epoch 4/12
469/469 [==============================] - 5s 10ms/step - loss: 0.6436 - accuracy: 0.7996 - val_loss: 0.3285 - val_accuracy: 0.9069
(train_mnist pid=3302450) Epoch 5/12
469/469 [==============================] - 5s 10ms/step - loss: 0.6433 - accuracy: 0.7976 - val_loss: 0.3262 - val_accuracy: 0.9071
(train_mnist pid=3302450) Epoch 6/12
469/469 [==============================] - 5s 11ms/step - loss: 0.6476 - accuracy: 0.8002 - val_loss: 0.3365 - val_acc

2023-12-05 02:00:18,216	INFO tune.py:1047 -- Total run time: 1244.61 seconds (1244.58 seconds for the tuning loop).


### Hyperband

In [ ]:
def train_mnist(config):
    num_classes = 10
    epochs = 12

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train.reshape(-1, 28, 28, 1) / 255.0, x_test.reshape(-1, 28, 28, 1) / 255.0
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=config["conv_filters"], kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(config["dropout"]),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"]),
        metrics=["accuracy"])

    for epoch in range(epochs):
        model.fit(
            x_train,
            y_train,
            batch_size=config["batch_size"],
            epochs=1,
            verbose=0,
            validation_data=(x_test, y_test))

        # Evaluate the model
        i, accuracy = model.evaluate(x_test, y_test, verbose=0)
        session.report({"mean_accuracy": accuracy})

In [ ]:
# Define the search space
search_space = {
    "conv_filters": tune.choice([64, 128, 256]),
    "lr": tune.loguniform(0.001, 0.1),
    "batch_size": tune.choice([64, 128, 256]),
    "dropout": tune.uniform(0, 1)
}

# Define the Hyperband scheduler
hyperband = HyperBandScheduler(
    time_attr="training_iteration",
    max_t=100,  # Maximum training iterations
    reduction_factor=3
)

# Run the optimization using Hyperband
start_time = time.time()

hyperband_analysis = tune.run(
    train_mnist,
    name="exp_hyperband",
    metric="mean_accuracy",
    mode="max",
    stop={"training_iteration": 12},  # Adjust based on your epochs
    resources_per_trial={"gpu": 1},
    config=search_space,
    num_samples=10,  # Number of different hyperparameter configurations to try
    scheduler=hyperband
)

end_time = time.time()
hyperband_time = end_time - start_time

2023-12-05 02:01:18,660	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(pid=3303162) 2023-12-05 02:01:20.480179: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=3303162) 2023-12-05 02:01:20.482465: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
(pid=3303162) 2023-12-05 02:01:20.529078: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
(pid=3303162) 2023-12-05 02:01:20.529611: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=3303162) To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3303162) 2023-12-05 02:01:2

Trial name,mean_accuracy
train_mnist_16ec0_00000,0.9626
train_mnist_16ec0_00001,0.956
train_mnist_16ec0_00002,0.9867
train_mnist_16ec0_00003,0.1009
train_mnist_16ec0_00004,0.9836
train_mnist_16ec0_00005,0.9869
train_mnist_16ec0_00006,0.9851
train_mnist_16ec0_00007,0.9577
train_mnist_16ec0_00008,0.9169
train_mnist_16ec0_00009,0.9845


(pid=3304567) 2023-12-05 02:04:39.216653: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=3304567) 2023-12-05 02:04:39.264370: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used. [repeated 2x across cluster]
(pid=3304567) 2023-12-05 02:04:39.264876: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(pid=3304567) To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3304567) 2023-12-05 02:04:40.193589: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(pid=3304567) /

**Answer:**

Above we can see the execution for Grid Search, Bayesian Search, and Hyperband for the given hyperparameter configurations. For each we have defined differet train_mnist functions since each of the methods has slightly different approach.

## 3.2

In [ ]:
print("Time taken for Grid Search: ", grid_time)

best_result = grid_analysis.best_result
print("Best mean accuracy found is: ", best_result["mean_accuracy"])

print("Best hyperparameters found were: ", grid_analysis.best_config)

Time taken for Grid Search:  5255.451504707336
Best mean accuracy found is:  0.9904000163078308
Best hyperparameters found were:  {'conv_filters': 256, 'lr': 0.001, 'batch_size': 64, 'dropout': 0.25}


In [ ]:
# Define the mapping for the categorical values
batch_size_map = {0: 64, 1: 128, 2: 256}

# Retrieve the best configuration
best_config = bayes_analysis.best_config

# Apply the mapping to the continuous values
best_config["conv_filters"] = round(best_config["conv_filters"])
best_config["batch_size"] = batch_size_map[int(round(best_config["batch_size"]))]

# Print the results
print("Time taken for Bayesian Search: ", bayes_time)
print("Best mean accuracy found is: ", bayes_analysis.best_result["mean_accuracy"])
print("Best hyperparameters found were: ", best_config)

Time taken for Bayesian Search:  617.1990833282471
Best mean accuracy found is:  0.9837999939918518
Best hyperparameters found were:  {'conv_filters': 105, 'lr': 0.01915704647548995, 'batch_size': 256, 'dropout': 0.18182496720710062}


In [ ]:
print("Time taken for Hyperband Search: ", hyperband_time)
print("Best mean accuracy found is: ", hyperband_analysis.best_result["mean_accuracy"])
print("Best hyperparameters found were: ", hyperband_analysis.best_config)

Time taken for Hyperband Search:  1284.8512477874756
Best mean accuracy found is:  0.9868999719619751
Best hyperparameters found were:  {'conv_filters': 64, 'lr': 0.0011908442812931575, 'batch_size': 128, 'dropout': 0.7741701209349096}


## 3.3

**Answer:**

Above outputs show us results of our 3 hyperparameter optimization methods - Grid Search, Bayesian Search, and Hyperband - on our Lenet model using the MNIST data. Looking at the outputs we can notice that all the models performed very well (maybe even too well - note the overfitting) with accuracies between 0.984 and 0.990. However, they had very different perfomances in the term of time taken to run it and hyperparameters they found and selected as best.

**Grid Search**
- First method we used was Grid Search. This method "tries out" **all** possible hyperparameter combinations of specified configurations and as such runs for the longest period of time. We can see that Grid Search took 5255.45 seconds to run but it did yield the highest accuracy of 0.9904. While this is the best accuracy compared to other 2 methods we need to consider if the long run time (and with that high computational cost) is worth it especially if we have a large number of potential hyperparameters.

**Bayesian Search**
- Next method we used was Bayesian Search. This method uses probabilistic modeling for parameter selection and due to that it is more efficient than Grid Search as it focuses on regions with more promising hyperparameters based on prior evaluations. We can see that Bayesian Search was the fastest of 3 methods and took only 617.20 seconds to run but it also achieved the lowest accuracy of 0.9838. In general, Bayesian Search can end in not optimal hyperparameters compared to Grid Search as it does not try all combinations but it is a good option as it is very fast and usually provides good results. While it did the worst out of our 3 methods it still achieved 0.98 accuracy which is very high.

**Hyperband Search**
- The final method we used was Hyperband Search. This method is resource efficiant optimization as it can very quickly identify promising hyperparameters by adaptively allocating resources.  Looking at the above results we can see that this method is a good choice if we want to find a middle ground between Grid Search and Bayesian Search as it has higher accuracy than Bayesian Search at 0.9869 (but lower than Grid Search) with much lower run time than Grid Search as it ran for 1284.85 seconds(but 2x longer than Bayesian Search).


Looking at all these we can see that while Grid Search provided the best accuracy it was the slowest method out of all. Further, while Bayesian Search ran fastest it had the lowest accuracy and Hyperband Search was somewhere in the middle on both accuracy and runtime front. In our case I would argue that we should use Bayesian Search becuase it ran fastest (and as such required the least computational resources) and it still achieved a very high accuracy of 0.98. However, in general we need to consider which is more important running quickly and using little computational resources or achiving the max accuracy (tradeoff between exploration and exploitation). If we are unsure, Hyperband Search is a good choice as it is somewhere between other two models.